In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/simplesentiment/products_sentiment_train.tsv
/kaggle/input/simplesentiment/products_sentiment_sample_submission.csv
/kaggle/input/simplesentiment/products_sentiment_test.tsv


In [125]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings('ignore')

In [126]:
df_train = pd.read_csv('/kaggle/input/simplesentiment/products_sentiment_train.tsv',header=None,sep='\t')
df_train.head()

,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [127]:
df_train.describe()

,1
count,2000.000000
mean,0.637000
std,0.480985
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [128]:
texts_train = df_train[0].values
texts_train

array(['2 . take around 10,000 640x480 pictures .',
       'i downloaded a trial version of computer associates ez firewall and antivirus and fell in love with a computer security system all over again .',
       'the wrt54g plus the hga7t is a perfect solution if you need wireless coverage in a wider area or for a hard-walled house as was my case .',
       ..., 'overall i like it . ',
       'i began taking pics as soon as i got this camera and am amazed at the quality of photos i have took simply by using the auto mode . ',
       "even after reading some of the instructions , it 's still hard to figure out . "],
      dtype=object)

In [129]:
labels_train = df_train[1].values
labels_train

array([1, 1, 1, ..., 1, 1, 0])

In [130]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [('vectorizer', vectorizer),
             ('classifier', classifier)
            ])

Протестируем разные классификаторы 

In [131]:
print(cross_val_score(text_classifier(CountVectorizer(),MultinomialNB()),
                      texts_train,labels_train,cv=5).mean())

0.776


In [132]:
print(cross_val_score(text_classifier(CountVectorizer(),LogisticRegression()),
                      texts_train,labels_train,cv=5).mean())

0.7685


In [133]:
print(cross_val_score(text_classifier(TfidfVectorizer(),LinearSVC()),
                      texts_train,labels_train,cv=5).mean())

0.7689999999999999


Подберем для каждого лучший по сетке

In [134]:
param_grid_NB = {'vectorizer__stop_words': [None, 'english'],
              'vectorizer__ngram_range': [(1,1), (1,2), (1,3), (1,4), (1,5)],
              'vectorizer__analyzer': ['word', 'char'],
              'vectorizer__min_df': [0.1,1, 10, 20],
                'classifier__alpha':[0,1,2]}
clf_bayes_CountV = GridSearchCV(text_classifier(CountVectorizer(),MultinomialNB()), param_grid_NB, n_jobs=-1)
clf_bayes_CountV = clf_bayes_CountV.fit(texts_train, labels_train)
print("Best score found by grid search:")
print(clf_bayes_CountV.best_score_)

Best score found by grid search:
0.7925000000000001


In [135]:
param_grid_LR = {'vectorizer__stop_words': [None, 'english'],
              'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
              'vectorizer__analyzer': ['word', 'char'],
              'vectorizer__min_df': [1, 10, 20],
                'classifier__penalty':['l2','l1'],
                'classifier__C':[0.2,1,2.3,3]}
clf_LR_CountV = GridSearchCV(text_classifier(CountVectorizer(),LogisticRegression()), param_grid_LR, n_jobs=-1)
clf_LR_CountV = clf_LR_CountV.fit(texts_train, labels_train)
print("Best score found by grid search:")
print(clf_LR_CountV.best_score_)

Best score found by grid search:
0.7729999999999999


In [136]:
param_grid_SVC_Tfidf = {'vectorizer__stop_words': [None, 'english'],
              'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
              'vectorizer__analyzer': ['word', 'char'],
              'vectorizer__min_df': [1, 10, 20],
                'classifier__penalty':['l2','l1'],
                'classifier__C':[0.2,1,2.3,3]}
clf_SVC_Tfidf = GridSearchCV(text_classifier(TfidfVectorizer(),LinearSVC()),
                             param_grid_SVC_Tfidf, n_jobs=-1)
clf_SVC_Tfidf = clf_SVC_Tfidf.fit(texts_train, labels_train)
print("Best score found by grid search:")
print(clf_SVC_Tfidf.best_score_)

Best score found by grid search:
0.793


Поэкспериментируем с различными ансамблями

In [137]:
param_grid = {'voting': ['hard'],
             'weights':[[1,1,1],[2,1,1],[1,2,1],[1,1,2]]}

clf_voting = GridSearchCV(VotingClassifier(estimators=[('NB', clf_bayes_CountV.best_estimator_), 
                                                  ('LR', clf_LR_CountV .best_estimator_),
                                                  ('SVC', clf_SVC_Tfidf.best_estimator_)
                                                 ]), param_grid, n_jobs=-1)
clf_voting = clf_voting.fit(texts_train, labels_train)
print("Best score found by grid search:")
print(clf_voting.best_score_)

Best score found by grid search:
0.7995000000000001


Применим балансировку классов

In [169]:
from imblearn.over_sampling import RandomOverSampler
over_sampler = RandomOverSampler(random_state=42)
X = texts_train.reshape(-1, 1)
y = labels_train
X_over, y_over = over_sampler.fit_sample(X,y)

Протестируем лучшие классификаторы на сбалансированных данных

In [170]:
print(cross_val_score(clf_voting.best_estimator_,X_over.reshape(1,X_over.shape[0])[0],y_over,cv=5).mean())

0.8563758234138449


In [172]:
print(cross_val_score(clf_SVC_Tfidf.best_estimator_,X_over.reshape(1,X_over.shape[0])[0],y_over,cv=5).mean())

0.8720844408490311


In [140]:
df_test = pd.read_csv('/kaggle/input/simplesentiment/products_sentiment_test.tsv',header=0,sep='\t')
df_test.head()
texts_test = df_test.text.values
ID_test = df_test.Id

In [168]:
clf5 = clf_voting.best_estimator_
clf5.fit(X_over.reshape(1,X_over.shape[0])[0],y_over)
y_pred5 = clf5.predict(texts_test)
ans5 = pd.DataFrame({'Id' : ID_test, 'y' : y_pred5})
ans5.head()
ans5.to_csv('submission8.csv', index=False)

In [173]:
clf6 = clf_SVC_Tfidf.best_estimator_
clf6.fit(X_over.reshape(1,X_over.shape[0])[0],y_over)
y_pred6 = clf6.predict(texts_test)
ans6 = pd.DataFrame({'Id' : ID_test, 'y' : y_pred6})
ans6.head()
ans6.to_csv('submission9.csv', index=False)

В итоге победила модель с линейным SVM,TFifd преобразованием и, что самое важное, с дополнительной балансировкой классов.